# ptyLab introduction (FP)

ptyLab is a highly modular coding package which can be used for both conventional and Fourier ptychography. Due to the modular nature it is easy to modify the code for various tasks and add your own functions, such as a new reconstruction engine or a calibration routine. To understand ptyLab we need to understand the basic classes contained within, which are briefly outlined below:


 -  ExperimentalData - this class is used to import the experimental data from an .hdf5 file. If the file contains the experimental images stored as an image stack (called ptychogram), probe/LED positions and several experimental parameters, then the data can be successfully imported and reconstructed.
 -  Reconstruction - this class creates various objects from the immutable ExperimentalData class which will be optimized/updated i.e. are mutable. 
 -  Params - this class stores parameters use for the reconstruction such that they could be exported/imported for different experiments
 -  Engines - all the engines used for the reconstruction are based on this class, which take the ExperimentalData and Optimizable objects as parameters and perform object/probe/pupil reconstruction.
 -  Monitor - visualization class used to display the reconstruction process
 -  CalibrationFPM - this package contains k-space position calibration routines caused by misaligned LED positions.


In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
from PtyLab.io import getExampleDataFolder
from PtyLab import Engines
import PtyLab

Cannot use pyqtgraph
Pyqtgraph not available
cupy not avialable
Cupy unavailable
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation
Cupy not available, will not be able to run GPU based computation


In [3]:
fileName = 'LungCarcinomaFPM.hdf5'
# fileName = 'USAFTargetFPM.hdf5'
filePath = getExampleDataFolder() / fileName

exampleData, reconstruction, params, monitor, engine, calib = PtyLab.easyInitialize(filePath, operationMode='FPM')

Copying attribute wavelength
Copying attribute dxd
Copying attribute zled
Copying attribute dxp


"Unable to open object (object 'orientation' doesn't exist)"


AttributeError: 'ExperimentalData' object has no attribute 'dxp'

## ExperimentalData class
The data must be stored as a ".hdf5" file, which enables structured file storage. For ptychographic data, experimental parameters such as wavelength or pixel size can be convieniently stored together with illumination/ecoder positions and the actual raw images/diffraction data in a single file.

A minimal list of fields required for ptyLab to work are:
- ptychogram - 3D image/diffraction data stack 
- wavelength - illumination lambda
- encoder - encoder positions / illumination angles
- dxd - detector pixel size
- zled - illumination-to-sample propagation distance
- magnification - magnification of the microscope

Also we have optional fields because they will either be computed later from the "required_fields" or are required for FPM, but not CPM (or vice-versa). If not provided by the user they will be set as None.
- NA -  numerical aperture of the microscope

The ".hdf5" file must contain a field called "ptychogram" containing the experimental raw images as a 3D array of shape [numFrames,X,Y], where numFrames is the number of images corresponding to each illumination vector in the "encoder" and X-Y are the 2D image dimensions. 

The ".hdf5" file must have a field called "encoder" containing the translation stage positions in units of meters (for CP) or the illumination angles in units of rad (for FP). The field "encoder" has a 2D shape [numFrames,2], where numFrames is the number of positions.

We start off our demonstration by creating the ExperimentalData() class which is used to load the .hdf5 file. In this example the variable "exampleData" will contain our class.

In [ ]:
mean_img = np.mean(exampleData.ptychogram,0)

## Reconstruction class
The ExperimentalData class contains immutable values. The Reconstruction class creates an object which will be mutable during the reconstruction or calibration procedure. It contains the reconstructed object and the probe/pupil. 

In FPM the intial object estimate can be computed from the raw data. "reconstruction.initialObject = 'upsampled'" will take the low-resolution raw data and create an upsampled object estimate via interpolation. The probe/pupil is set to be a clear circle representing a fully transparent aperture without any aberration. Once everything is set use "initializeObjectProbe()" method.

In [ ]:
reconstruction.initialProbe = 'circ'
reconstruction.initialObject = 'upsampled'
reconstruction.initializeObjectProbe()

## Monitor class
This class will create a monitor to visualize the reconstruction. 

In [ ]:
# Set monitor properties
monitor.figureUpdateFrequency = 10
monitor.objectPlot = 'complex'  
monitor.verboseLevel = 'low'
monitor.objectPlotZoom = .01 
monitor.probePlotZoom = .01 

## Params class
This class holds various switches and parameters that can be specified before a reconstruction is carried out. For instance to determine whether the reconstruction is carried out on CPU or GPU, or to specify the order of probe positions to be random or sequential.

In [ ]:
params.gpuSwitch = True
params.positionOrder = 'NA'
params.probeBoundary = True
params.adaptiveDenoisingSwitch = True

## IlluminationCalibration class
Calibration class based on "Regina Eckert, Zachary F. Phillips, and Laura Waller, "Efficient illumination angle self-calibration in Fourier ptychography," Appl. Opt. 57, 5434-5442 (2018)"

In [ ]:
calib.plot = True
calib.fit_mode ='SimilarityTransform'
calib.calibrateRadius = True
calib.runCalibration()

## Engine class
A specific engine can be imported (e.g. ePIE, mPIE etc.) and used on the reconstruction class to optimize our initial estimates for the object/probe. In this example we use the quasi-Newton method to reconstruct the data, by passing the reconstruction, exampleData and monitor objects to be used during the reconstruction

In [ ]:
engine = Engines.mqNewton(reconstruction, exampleData, params, monitor)
engine.reconstruct()

In [ ]:
from PtyLab.utils.utils import fft2c
from PtyLab.utils.visualisation import modeTile
plt.rcParams['figure.figsize'] = [12, 12]
plt.rcParams['figure.dpi'] = 100 

plt.subplot(121)
plt.title('raw data')
plt.imshow(abs(mean_img),cmap='gray')
plt.subplot(122)
plt.title('reconstruction')
plt.imshow(abs(modeTile(fft2c(reconstruction.object))),cmap='gray')

In [ ]:
# reconstruction.saveResults('results')